# Dataset: *ottawashooting*

- df.to_csv(r'D:\論文\PHEME9\Data\CSV\ottawashooting-df.csv',index=False)
- src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\ottawashooting-src.csv',index=False)

In [1]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSequenceClassification

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\ottawashooting-df.csv', encoding = 'utf-8', header = 0)
src_tw_df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\ottawashooting-src.csv', encoding = 'utf-8', header =0)
print(df.shape)
print(src_tw_df.shape)

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace = True)
df.reset_index(drop=True,inplace=True)

In [ ]:
src_tw_df.isnull().sum()

In [12]:
src_tw_df[src_tw_df.cleaned_src_tw.isnull()].src_tweet

62     RT @ctvmercedes: Ottawa: don't tweet your loca...
64     RT @bts_bighit: [Episode] #BTS 'War of Hormone...
182    RT @scottbix: Incredible shot: A woman gives m...
386    RT @cfedio: Being told at CBC Ottawa that "sit...
428    RT @WatchCTVNews: LIVE NOW: Cops swarm Parliam...
515    RT @CBCOttawa: CONFIRMED | 1 shooter shot dead...
747    RT @ArmedResearch:#ISIS Media account posts pi...
Name: src_tweet, dtype: object

In [13]:
a = src_tw_df[src_tw_df.cleaned_src_tw.isnull()].src_tweet_id.values
df.loc[df.src_tweet_id.isin(a),['reply_tweet','cleaned_reply_tw']]

,reply_tweet,cleaned_reply_tw
779,@CTVNews @CTVMercedes lol,lol
780,@CTVNews @CTVMercedes and make sure shooters a...,make sure shooters not take alive
781,@CTVNews @davidcollict @CTVMercedes fast to be...,fast responsible statement news station wish w...
782,@CTVNews @Peady @CTVMercedes I guess it's not ...,guess not safe patio
783,@oldbakerman @CTVNews @CTVMercedes Stay inside...,stay inside stay inside
...,...,...
8717,@GeatanD @R_Mallough @ArmedResearch what a pie...,piece shit bad take someone
8718,@GeatanD @charlesadler @ArmedResearch do not u...,not use name post pic doesnt deserve
8719,@GeatanD @charlesadler well Mr. Trudeau should...,well mr trudeau still go offer bottle maple sy...
8720,@GeatanD @charlesadler good riddance,good riddance


## Emotion Analysis

### T5 Based
<https://huggingface.co/mrm8488/t5-base-finetuned-emotion?text=I+wish+you+were+here+but+it+is+impossible>

In [ ]:
t5_tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion",use_fast=False)
t5_model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion")



def get_emotion_t5(text):
    input_ids = t5_tokenizer.encode(text , return_tensors='pt')
    output = t5_model.generate(input_ids=input_ids,max_length=2)
  
    dec = [t5_tokenizer.decode(ids) for ids in output]
    label = dec[0]
    return label

In [ ]:
emotion_t5 = df.cleaned_reply_tw.apply(get_emotion_t5)

In [ ]:
df['reply_emo_t5'] = emotion_t5

In [ ]:
import re
df['reply_emo_t5'] = df['reply_emo_t5'].apply(lambda x : re.sub('<pad> ',"",x))

In [ ]:
src_emo_t5 = src_tw_df.cleaned_src_tw.astype(str).apply(get_emotion_t5)
src_tw_df['src_emo_t5'] = src_emo_t5

In [ ]:
src_tw_df['src_emo_t5']  = src_tw_df.src_emo_t5.apply(lambda x : re.sub('<pad> ',"",x))

### roBERTa-base model
<https://huggingface.co/cardiffnlp/twitter-roberta-base-emotion>

In [5]:
import numpy as np
from scipy.special import softmax

task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
ro_tokenizer = AutoTokenizer.from_pretrained(MODEL)
ro_model = AutoModelForSequenceClassification.from_pretrained(MODEL)


In [6]:
def get_emotion_roBERT(text):
    
    def preprocess(txt):
        new_text = []
        for t in txt.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        return " ".join(new_text)
    
    labels = ['anger','joy','optimism','sadness']
    text = preprocess(text)
    encoded_input = ro_tokenizer(text, return_tensors='pt')
    output = ro_model(**encoded_input)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    emo = labels[ranking[0]]
    return emo

In [7]:
emotion_ro = df.cleaned_reply_tw.apply(get_emotion_roBERT)

In [8]:
df['reply_emo_ro'] = emotion_ro

In [10]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\ottawashooting-df-2.csv',index=False)

In [9]:
src_emo_ro = src_tw_df.cleaned_src_tw.astype(str).apply(get_emotion_roBERT)
src_tw_df['src_emo_ro'] = src_emo_ro

In [11]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\ottawashooting-src-2.csv',index=False)

 - robert : 'anger','joy','optimism','sadness'
 - t5: 'sadness', 'joy', 'love', 'anger', 'fear', 'surprise' 
 - text2emotion: 'angry, 'fear', 'happy, 'sad', 'surprise'

In [ ]:
print("original reply tw: ", df.reply_tweet[8])
print("cleaned reply tw: ", df.cleaned_reply_tw[8])

print('roBERT based: ',df.reply_emo_ro[8])
print('t5 based: ',df.reply_emo_t5[8])
print('text2emotion: ', df.reply_emo_te[8])